# **知识图谱构建**
大致分为以下几步

1、数据预处理

2、切分文段

3、输入大模型

4、反馈整合输出

5、分步重构

## **数据预处理**

目标：在不使用ocr的情况下，将文件转成合适长度的文本段落

数据源：规整的docx类型，以及内容为图像的pdf文件

开源库：**python-docx**，内容按段落切分，没有“页”以及“目录”的操作api

**PyMuPDF**，内容按页切分，输入文件需为含有文本信息的pdf文件

处理方式：将docx转成pdf然后按照pdf进行数据读取和按页切分

In [1]:
# 将目录数据转成实体关系数据以及切片数据
from buildKG.str2json import indexstr2json,str2json
from buildKG.docx2request import group_paragraphs_by_token_limit,extract_text_from_pages,group_page_by_token_limit
from buildKG.forest import Node,Edge,Forest

from http import HTTPStatus
from dashscope import Generation
from dashscope.api_entities.dashscope_response import Role
import dashscope
import time
import random
import os
import fitz  # PyMuPDF
import copy
dashscope.api_key = "sk-8cdfdc6958a7408492d465269349299f"

下面不好用，要windows系统，我直接wps转pdf,当然不转直接拿langchain的Unstructured来做也可以，同样问题也有不如WPS docx转pdf 人工选取 目录页数快

In [2]:
# # docx转pdf 
# from docx2pdf import convert

# # 指定.docx文件路径和.pdf输出路径
# docx_file_path = './data/（139） 3D打印技术基础（第三版）-朱红,易杰,谢丹-华中科技大学出版社.docx'
# pdf_output_path = './data_pdf/（139） 3D打印技术基础（第三版）-朱红,易杰,谢丹-华中科技大学出版社.pdf'

# # 执行转换
# convert(docx_file_path, pdf_output_path)

## **文段切分**
目标：选取合适大小的文本段落为单元任务，去让大模型进行归纳总结

1、直接按长度进行划分：很碎，造成野节点非常多，不能作为一个方案

2、按章节进行划分：不够细致，大多数只挺留在了第一级

3、按小节进行划分：很深，很杂，会把很多一个定义下的大段文本作为输出，同样野节点非常多

处理方式：

按章节进行划分的方式，以章节名作为一级子节点，后续野子节点置于一级子节点上

In [9]:
# 目录起始页
# 若篇幅过长，则手动切分目录，对目录进行分批处理
start_page = 3
end_page = 4
pdf_output_path = './pdf/面向对象.pdf'

In [10]:
# 执行目录提取
request=extract_text_from_pages(pdf_output_path, start_page, end_page)

**将切分好的目录数据给到大模型，让大模型总结目录**

In [11]:
nodes_array=[]
edges_array=[]
pages_array=[]
pages_part_array=[]
chapter_name_array=[]
log=[]

In [12]:
prompt = """
    在接下来的对话中，你需要遵循以下规则：
    你是一台用于处理文本数据的AI系统。你的任务是处理我上传的文本文件，并按照以下要求进行操作：
    - 输出标准JSON格式的切片结果。
    - 剔除文件中的所有乱码（乱码不包括英文）。
    - 处理英文的时候 请保留英文的原格式，包括括号。
    - 根据你的判断，可以筛选无意义的、与目录无关的句子。但是要确保没有改变原意，保留了原本的信息量。
    - 确保没有信息遗漏，不要缺省任何有效信息。
    - 按照章节、小节，对内容进行切片，构建树状JSON结构，请确保你输出的JSON的格式规范。
    - 请严格参考格式示例:
    {
      "slices": [
        {
      "chapter_name": "(章节去除编号对应名称）",
      "page": "（章节对应页码）",
      "entries": [
        {
          "entry_name": "（小节去除编号对应名称）",
          "page": "（小节对应页码）",
        }
        {
          "entry_name": "（小节去除编号对应名称）",
          "page": "（小节对应页码）",
        }
      ]
    },
      ]
    }
    - 确保每一个条目都被完整地处理，不漏切片。
    - 对chapter_name、entries中的属性，请不要带上任何标号性质的前缀，并对你提取到的名称进行概括
    - 不要输出sub_entries
    - 你不需要确认收到，请从第一条讯息开始，直接开始生成完整准确的输出，不允许输出json之外的内容
    - 不要输出json格式之外的部分
"""

messages = [{'role': Role.SYSTEM, 'content': prompt},
            {'role': Role.USER, 'content': request}]
response = Generation.call(
    Generation.Models.qwen_max_longcontext,
    messages=messages,
    result_format='message',  # set the result to be "message" format.
)
if response.status_code == HTTPStatus.OK:
    # append result to messages.
    raw_string=response.output.choices[0]['message']['content']
    print(raw_string)
    # 节点，边，页数，小节名称，章节名称
    nodes_array,edges_array,pages_array,pages_part_array,chapter_name_array=indexstr2json(raw_string)
else:
    print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
        response.request_id, response.status_code,
        response.code, response.message
    ))

```json
{
  "slices": [
    {
      "chapter_name": "数据库设计",
      "page": "213",
      "entries": [
        {
          "entry_name": "逻辑模型和物理模型",
          "page": "213"
        },
        {
          "entry_name": "关系数据库模型",
          "page": "215"
        },
        {
          "entry_name": "将 E-R 图转化成关系",
          "page": "216"
        },
        {
          "entry_name": "设计字段",
          "page": "222"
        },
        {
          "entry_name": "代码设计",
          "page": "225"
        }
      ]
    },
    {
      "chapter_name": "输出、输入与用户界面设计",
      "page": "230",
      "entries": [
        {
          "entry_name": "输出设计",
          "page": "230"
        },
        {
          "entry_name": "输入设计",
          "page": "233"
        },
        {
          "entry_name": "用户界面设计",
          "page": "241"
        },
        {
          "entry_name": "输入输出界面示例",
          "page": "251"
        }
      ]
    },
    {
      "chapter_name": "系统实现与运行",
      "page": "256",
      "entri

**章节列表**

In [13]:
# 检查LLM输出是否带有章节号，是否带有小节等会污染节点的内容
part_array=copy.copy(nodes_array)
print(len(part_array))

46


In [19]:
# 结果保存
import pickle
# 节点
with open('node.pkl', 'wb') as f:
    pickle.dump(nodes_array, f)
# 边
with open('edge.pkl', 'wb') as f:
    pickle.dump(edges_array, f)
# 报错日志
with open('log.pkl', 'wb') as f:
    pickle.dump(log, f)
# 章节加小节列表
with open('part_array.pkl', 'wb') as f:
    pickle.dump(part_array, f)
# 小节页数
with open('pages_array.pkl', 'wb') as f:
    pickle.dump(pages_array, f)
# 小节列表
with open('pages_part_array.pkl', 'wb') as f:
    pickle.dump(pages_part_array, f)
# 章节名称
with open('chapter_name_array.pkl','wb')as f:
    pickle.dump(chapter_name_array, f)
# 章节与小节之间的边关系
with open('chapter_edge_array.pkl','wb')as f:
    pickle.dump(edges_array, f)

['信息系统开发概述',
 '信息系统开发路线、方法与工具',
 '信息系统项目管理',
 '需求获取',
 '过程建模',
 '数据建模',
 '应用架构设计',
 '软件过程设计',
 '数据库设计',
 '输出、输入与用户界面设计',
 '系统实现与运行',
 '面向对象开发概述',
 'UML',
 '面向对象需求理解',
 '面向对象分析',
 '面向对象设计',
 '面向对象实现']

In [14]:
# 如果页数过多，目录无法实现一次生成可以使用这个做分批处理
import pickle
with open('node.pkl', 'rb') as f:
    nodes_array = pickle.load(f)+nodes_array
with open('edge.pkl', 'rb') as f:
    edges_array = pickle.load(f)+edges_array
with open('log.pkl', 'rb') as f:
    log = pickle.load(f)+log
with open('part_array.pkl', 'rb') as f:
    part_array = pickle.load(f)+part_array
with open('pages_array.pkl', 'rb') as f:
    pages_array = pickle.load(f)+pages_array
with open('pages_part_array.pkl', 'rb') as f:
    pages_part_array = pickle.load(f)+pages_part_array
with open('chapter_name_array.pkl', 'rb') as f:
    chapter_name_array = pickle.load(f)+chapter_name_array
with open('chapter_edge_array.pkl', 'rb') as f:
    chapter_edge_array=pickle.load(f)+edges_array

['信息系统开发概述',
 '信息系统开发路线、方法与工具',
 '信息系统项目管理',
 '需求获取',
 '过程建模',
 '数据建模',
 '应用架构设计',
 '软件过程设计',
 '数据库设计',
 '输出、输入与用户界面设计',
 '系统实现与运行',
 '面向对象开发概述',
 'UML',
 '面向对象需求理解',
 '面向对象分析',
 '面向对象设计',
 '面向对象实现']

In [3]:
# 重新加载
import pickle
with open('node.pkl', 'rb') as f:
    nodes_array = pickle.load(f)
with open('edge.pkl', 'rb') as f:
    edges_array = pickle.load(f)
with open('log.pkl', 'rb') as f:
    log = pickle.load(f)
with open('part_array.pkl', 'rb') as f:
    part_array = pickle.load(f)
with open('pages_array.pkl', 'rb') as f:
    pages_array = pickle.load(f)
with open('pages_part_array.pkl', 'rb') as f:
    pages_part_array = pickle.load(f)
with open('chapter_name_array.pkl', 'rb') as f:
    chapter_name_array = pickle.load(f)
with open('chapter_edge_array.pkl', 'rb') as f:
    chapter_edge_array=pickle.load(f)


**对应页数**

In [16]:
# 这段代码不能重复跑，pages_array需要重新生成

# 对于一些目录页数不参与文件内页数的我们可以执行下面这两行
# pages_array=[page+end_page for page in pages_array]
# pages_array=sorted(pages_array)
# 补充尾页信息
# 打开 PDF 文件
pdf_document = fitz.open(pdf_output_path)
    # 获取总页数
total_pages = pdf_document.page_count
pages_array.append(total_pages)

## **3.输入提示词设计**

目标：指引大模型更好的利用资源优势实现按照固定格式的输出

携带上下文：影响输出格式，输出很杂，会导致实体中有大段文本出现

不携带上下文：不够细致

改进方向：
围绕目录信息，将章节内部信息给到大模型，让大模型去执行抽取任务

In [26]:

node_array=copy.copy(part_array)
debug_array=[]
part_name=""
prompt = f"""
    在接下来的对话中，你需要遵循以下规则：
    你是一个擅长读书并总结的人，你的任务是处理我上传的课本选段，并按照以下要求进行操作：
    - 输出标准JSON格式字符串。
    - 剔除文件中的所有乱码（乱码不包括英文）。
    - 处理英文的时候 请保留英文的原格式，包括括号。
    - 我会告诉你这段文字的主题，你可以围绕着这个主题开始一层一层地梳理知识结构
    - 你需要对文本内知识点进行提取，并列成列表，形成json字符串的point字段。
    - 当你提取到一个知识点，在上下文寻找与他相关联的知识点，并进行提取
    - 你需要对知识点与知识点之间的属于关系进行提取，将有属于关系的知识点两两之间形成(知识点1,属于，知识点2)的元组，并以列表的形式存进json字符串的edge字段
    - 像"一"、"二"或是"第一"、"第二"等形式的序号字符，可用于帮助定位实体。标题之间的层级结构，也可以帮助构建属于关系。
    - 这一节的名字叫{part_name},请围绕{part_name}进行知识点提取和属于关系识别
    - 格式参考示例:
    {{
        "point": ["数学","线性代数","矩阵"],
        "edge":[["线性代数","属于","数学"],["矩阵","属于","线性代数"]]
    }}
    - 请保证仅提取属于关系，被提取出的知识点一定有相应的关系。
    你不需要确认收到，或是做出过多解释，直接开始生成完整准确的输出。
"""
    # 下面这段原本是写在提示词里面的，表征上下文，但是由于我们将提取目录中，构建关系的部分删去了，这部分写入提示词会导致边生成不出来，故注释掉
    # - 同时你可以参考上下文中已经提取好的实例和关系,在生成答案的时候，你必须保证这些实例和关系不能改变，只能在这些基础之上新增实体和关系：
    # {{
    #     "point": {nodes_array},
    #     "edge":  {edges_array}
    # }}
    # 实测加了这段话生成的内容边多了，但是很杂，影响最后生成结果且会造成各种生成的答案格式不正确等问题；但是理论上来说，我们需要上下文提示，下一期工程再改吧
for i in range(0,len(pages_array)-1):
    print("————————————第"+str(i+1)+"章————————————")
    start=pages_array[i]
    end=pages_array[i+1]
    part_name=pages_part_array[i]
    prompt = f"""
    在接下来的对话中，你需要遵循以下规则：
    你是一个擅长读书并总结的人，你的任务是处理我上传的课本选段，并按照以下要求进行操作：
    - 输出标准JSON格式字符串。
    - 剔除文件中的所有乱码（乱码不包括英文）。
    - 处理英文的时候 请保留英文的原格式，包括括号。
    - 我会告诉你这段文字的主题，你可以围绕着这个主题开始一层一层地梳理知识结构
    - 你需要对文本内知识点进行提取，并列成列表，形成json字符串的point字段。
    - 当你提取到一个知识点，在上下文寻找与他相关联的知识点，并进行提取
    - 你需要对知识点与知识点之间的属于关系进行提取，将有属于关系的知识点两两之间形成(知识点1,属于，知识点2)的元组，并以列表的形式存进json字符串的edge字段
    - 像"一"、"二"或是"第一"、"第二"等形式的序号字符，可用于帮助定位实体。标题之间的层级结构，也可以帮助构建属于关系。
    - 这一节的名字叫{part_name},请围绕{part_name}进行知识点提取和属于关系识别
    - 格式参考示例:
    {{
        "point": ["数学","线性代数","矩阵"],
        "edge":[["线性代数","属于","数学"],["矩阵","属于","线性代数"]]
    }}
    - 请保证仅提取属于关系，被提取出的知识点一定有相应的关系。
    你不需要确认收到，或是做出过多解释，直接开始生成完整准确的输出。
    """
    # 50000是token数量，可以用来选择单元切片的大小
    # 经过我的测试，小节所表现出的效果不如章节，很乱很杂，当然也有可能是因为上下文提示词的原因
    contents=group_page_by_token_limit(pdf_output_path,50000, start, end)
    for content_id,content in enumerate(contents):
        request=""
        for paragraph_id,paragraph in enumerate(content):
            request+=paragraph
            request+='\n'
        # messages.append({'role': Role.USER, 'content': request})
        messages = [{'role': Role.SYSTEM, 'content': prompt},
                {'role': Role.USER, 'content': request}]
        response = Generation.call(
            Generation.Models.qwen_max_longcontext,
            messages=messages,
            result_format='message',  # set the result to be "message" format.
        )
        if response.status_code == HTTPStatus.OK:
            raw_string=response.output.choices[0]['message']['content']
            print(raw_string)
            print(response.usage["total_tokens"])
            # append result to messages.
            nodes_array_n,edges_array_n=str2json(raw_string)
            nodes_array+=[node for node in nodes_array_n if node not in nodes_array]
            edges_array+=[edge for edge in edges_array_n if edge not in edges_array]
            log.append((i,nodes_array_n,edges_array_n))
            if nodes_array_n ==[] or edges_array_n==[]:
                debug_array.append(i)
        else:
            print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
                response.request_id, response.status_code,
                response.code, response.message
            ))
            debug_array.append(i)
            break
        # api限制是1分钟调用一次
        secs = random.normalvariate(38, 0.4)
        if secs <= 30 or secs >= 45:
            secs =40   # 太小或太大则重置为平均值
        print("sleep:"+str(secs)+"秒")
        time.sleep(secs)       

————————————第1章————————————
{
  "point": [
    "系统",
    "系统的基本概念",
    "系统的目标",
    "系统的边界",
    "构件",
    "构件之间的关系",
    "环境",
    "接口",
    "输入",
    "输出",
    "约束",
    "信息系统",
    "信息系统的创始人",
    "Gordon B. Davis",
    "信息系统定义",
    "薛华成教授的观点",
    "信息系统特征",
    "人机信息系统",
    "企业管理信息系统",
    "信息技术",
    "信息技术定义",
    "信息和通信技术",
    "信息技术的应用",
    "计算机硬件和软件",
    "网络和通讯技术",
    "应用软件开发工具",
    "信息系统在商业中的角色",
    "基于信息技术的组织管理方案",
    "信息技术体系结构",
    "企业战略目标",
    "系统方法",
    "系统方法作为方法论",
    "整体性",
    "结构性",
    "关联性",
    "功能性",
    "信息系统的功能结构",
    "输入功能",
    "存储功能",
    "处理功能",
    "输出功能",
    "控制功能"
  ],
  "edge": [
    ["系统", "属于", "系统的基本概念"],
    ["系统的目标", "属于", "系统的基本概念"],
    ["系统的目标", "是", "达到某种目的"],
    ["系统的目标", "举例", "快餐店系统的主要目标"],
    ["系统边界", "属于", "系统的基本概念"],
    ["构件", "属于", "系统的基本概念"],
    ["构件之间的关系", "属于", "系统的基本概念"],
    ["环境", "属于", "系统的基本概念"],
    ["接口", "属于", "系统的基本概念"],
    ["输入", "属于", "系统的基本概念"],
    ["输出", "属于", "系统的基本概念"],
    ["约束", "属于", "系统的基本概念"],
  

## **4.反馈整合，xmind输出**

构建了森林、边、节点类，用节点表征实体，用边表征属于关系，relationship表征其他关系
遇到问题：
1、有的边添加时间过长，但是从边上看暂未发现原因
2、只添加其他关系不添加从属关系不能使节点被显示，我们在其他关系上也加了一个从属关系这会导致从属混乱，程序执行出错

In [5]:
# 构建xmind
forest = Forest()
forest.build_forest(chapter_name_array,[],None)
for chapter in chapter_name_array:
    chapter_edges=[edge for edge in chapter_edge_array if edge[2]==chapter]
    chapter_nodes=[edge[0] for edge in chapter_edges]
    forest.build_forest(chapter_nodes,chapter_edges,chapter)
for l in log:
    id,nodes_array_i,edges_array_i=l
    part=part_array[id]
    print(part)
    forest.build_forest(nodes_array_i,edges_array_i,part)
forest.build_kg("面向对象")

('信息系统的基本概念', '属于', '信息系统开发概述')
('信息系统的结构', '属于', '信息系统开发概述')
('信息系统开发过程', '属于', '信息系统开发概述')
('信息系统的相关角色', '属于', '信息系统开发概述')
('本章小结', '属于', '信息系统开发概述')
('案例分析——校园二手书交易平台的设计与实现', '属于', '信息系统开发概述')
('信息系统开发路线概述', '属于', '信息系统开发路线、方法与工具')
('模型驱动开发路线', '属于', '信息系统开发路线、方法与工具')
('快速应用开发路线', '属于', '信息系统开发路线、方法与工具')
('商用软件包开发路线', '属于', '信息系统开发路线、方法与工具')
('选择合适的开发路线与方法', '属于', '信息系统开发路线、方法与工具')
('自动化工具与技术', '属于', '信息系统开发路线、方法与工具')
('案例：CASE 工具在系统开发中的应用', '属于', '信息系统开发路线、方法与工具')
('本章小结', '属于', '信息系统开发路线、方法与工具')
('信息系统项目管理生命周期', '属于', '信息系统项目管理')
('项目组织', '属于', '信息系统项目管理')
('项目管理技术', '属于', '信息系统项目管理')
('案例：某系统开发项目管理', '属于', '信息系统项目管理')
('本章小结', '属于', '信息系统项目管理')
('案例分析', '属于', '信息系统项目管理')
('系统需求概述', '属于', '需求获取')
('需求获取过程', '属于', '需求获取')
('需求获取的方法', '属于', '需求获取')
('案例分析——客户关系管理系统的需求获取', '属于', '需求获取')
('本章小结', '属于', '需求获取')
('过程建模', '属于', '过程建模')
('数据流图', '属于', '过程建模')
任务执行超时
('2.0 层数据流图', '属于', '过程建模')
任务执行超时
('过程逻辑', '属于', '过程建模')
任务执行超时
('数据字典', '属于', '过程建模')
任务执行超时
('过程建模步骤', '属于', '过程建模')
任务执行

KeyboardInterrupt: 

## **5.分步重构**
主要是两方面的工作：
1.选段重新输入大模型
2.大模型生成结果保存

**大模型生成结果保存**

In [27]:
import pickle
with open('node.pkl', 'wb') as f:
    pickle.dump(nodes_array, f)
with open('edge.pkl', 'wb') as f:
    pickle.dump(edges_array, f)
with open('log.pkl', 'wb') as f:
    pickle.dump(log, f)
with open('part_array.pkl', 'wb') as f:
    pickle.dump(part_array, f)

In [4]:
import pickle
with open('node.pkl', 'rb') as f:
    nodes_array = pickle.load(f)
with open('edge.pkl', 'rb') as f:
    edges_array = pickle.load(f)
with open('log.pkl', 'rb') as f:
    log = pickle.load(f)
with open('part_array.pkl', 'rb') as f:
    part_array = pickle.load(f)

**选段重新输入大模型**

In [29]:
# 这块主要是为了处理出现错误的流程,当然如果觉得生成效果不够理想，也可以通过更改debug_array去将选中章节添加入重新生成的段落
# 错误出现的主要原因是：1、文段过长，2、输出格式出错，3、请求过快或网络连接不稳定，4、请求文段中含有敏感文段
# debug_array=[4, 9, 11, 13, 17, 25]
debug_array=list(set(debug_array))
print(debug_array)

[81, 58, 60, 68]


In [8]:
debug_id=0
while debug_id<len(debug_array):
    i=debug_array[debug_id]
    start=pages_array[i]
    end=pages_array[i+1]
    contents=group_page_by_token_limit(pdf_output_path,50000, start, end)
    debug_state=True
    for content_id,content in enumerate(contents):
        request=""
        print("————————————第"+str(content_id)+"次请求————————————")
        for paragraph_id,paragraph in enumerate(content):
            request+=paragraph
            request+='\n'
        # messages.append({'role': Role.USER, 'content': request})
        messages = [{'role': Role.SYSTEM, 'content': prompt},
                {'role': Role.USER, 'content': request}]
        response = Generation.call(
            Generation.Models.qwen_max_longcontext,
            messages=messages,
            result_format='message',  # set the result to be "message" format.
        )
        if response.status_code == HTTPStatus.OK:
            raw_string=response.output.choices[0]['message']['content']
            print(raw_string)
            print(response.usage["total_tokens"])
            # append result to messages.
            nodes_array_n,edges_array_n=str2json(raw_string.replace("'", '"'))
            nodes_array+=[node for node in nodes_array_n if node not in nodes_array]
            edges_array+=[edge for edge in edges_array_n if edge not in edges_array]
            log.append((i,nodes_array_n,edges_array_n))
        else:
            print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
                response.request_id, response.status_code,
                response.code, response.message
            ))
            debug_state=False
            break
        secs = random.normalvariate(80, 0.4)
        if secs <= 60 or secs >= 120:
            secs =80   # 太小则重置为平均值
        print("sleep:"+str(secs)+"秒")
        time.sleep(secs)
    if debug_state:
        debug_id+=1

————————————第0次请求————————————


NameError: name 'prompt' is not defined